In [ ]:
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [ ]:
# Define data directories and parameters
train_dir = "/media/ailama/HDD Storage1/users/ailama/Banpu/train"
val_dir = "/media/ailama/HDD Storage1/users/ailama/Banpu/valid"
test_dir = "/media/ailama/HDD Storage1/users/ailama/Banpu/test"
img_size = (224, 224)  # Adjust to your model's input size
batch_size = 32

In [ ]:
# Create a new ImageDataGenerator for data augmentation
datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

In [ ]:
val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

In [ ]:
# Define a pre-trained model
base_model = MobileNetV2(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# Compile the model
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(train_generator,
          epochs=20,  # Increase the number of epochs
          steps_per_epoch=len(train_generator),
          validation_data=val_generator,
          validation_steps=len(val_generator))

In [ ]:
# Evaluate the model on the test set
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

In [ ]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

In [ ]:
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

In [ ]:
model.save("test_7.h5")

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np
from tensorflow.keras.models import load_model

img_size = (224, 224)

# Load the single image
# image_path = "/media/ailama/HDD Storage1/users/ailama/Banpu/data/original_bad/17.jpg"
image_path = "/media/ailama/HDD Storage1/users/ailama/Banpu/data/original_bad/123.jpg"
# image_path = "/media/ailama/HDD Storage1/users/ailama/Banpu/data/original_good/86.png"

img = image.load_img(image_path, target_size=img_size)
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0  # Normalize pixel values to the range [0, 1]

loaded_model = load_model("test_7.h5")

# Use the loaded model for prediction
prediction = loaded_model.predict(img_array)
print(prediction)

# Interpret the prediction
predicted_class = "good weld" if prediction > 0.8 else "bad weld"
print("Predicted class:", predicted_class)